In [0]:
account_name = "goodreadsreviews60313569"
account_key="TLLUqPCW94aCOPP/8ic0S/0oWOEa9ccZ/rfHpUw71lh9c2vsCCl2oFGaWifNoAmtRTC45MDRTn0B+AStXqUGDQ=="  # the long key from the portal / lab

spark.conf.set(
    f"fs.azure.account.key.{account_name}.dfs.core.windows.net",
    account_key
)



In [0]:
gold_df = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60313569.dfs.core.windows.net/gold/feature_v2/train"
)


In [0]:
%pip install nltk

import nltk
# Other code that uses nltk follows...

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# Install Hugging Face Transformers and Torch.
# IMPORTANT: After running this cell, restart your cluster or re-run all cells to ensure the libraries are available before any import statements.
%pip install transformers torch

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install emoji

import emoji

# Example usage of the emoji library
text_with_emoji = "Hello, world! 🌍"
cleaned_text = emoji.replace_emoji(text_with_emoji, replace='')  # Remove emojis
print(cleaned_text)

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Hello, world! 


In [0]:
%sh
az storage container create \
  --name lakehouse \
  --account-name <goodreadsreviews60313569> \
  --auth-mode login


bash: line 1: goodreadsreviews60313569: No such file or directory


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
import os

# TODO: Update the schema below to match your actual data columns and types
schema = StructType([
    StructField("id", StringType(), True),
    StructField("review_text", StringType(), True),
    StructField("rating", IntegerType(), True),
    StructField("user_id", StringType(), True),
    StructField("book_id", StringType(), True),
    # Add more fields as needed
])

parquet_path = 'abfss://lakehouse@goodreadsreviews60313569.dfs.core.windows.net/gold/feature_v2/'

try:
    df = spark.read.schema(schema).parquet(parquet_path)
    if df.rdd.isEmpty():
        print(f"WARNING: Parquet path {parquet_path} is empty. Creating empty DataFrame with schema.")
        df = spark.createDataFrame([], schema)
except Exception as e:
    print(f"WARNING: Could not read Parquet path {parquet_path}. Creating empty DataFrame with schema. Error: {e}")
    df = spark.createDataFrame([], schema)

In [0]:
base_path = "abfss://lakehouse@goodreadsreviews60313569.dfs.core.windows.net/gold/feature_v2"


In [0]:
storage_account = "goodreadsreviews60313569"
container = "lakehouse"

account_key = "TLLUqPCW94aCOPP/8ic0S/0oWOEa9ccZ/rfHpUw71lh9c2vsCCl2oFGaWifNoAmtRTC45MDRTn0B+AStXqUGDQ=="  # example format

spark.conf.set(
    f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
    account_key
)

base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/gold/feature_v2"

print("storage_account =", storage_account)
print("base_path      =", base_path)


storage_account = goodreadsreviews60313569
base_path      = abfss://lakehouse@goodreadsreviews60313569.dfs.core.windows.net/gold/feature_v2


In [0]:
# Import necessary libraries
try:
    from transformers import BertTokenizer, BertModel
    import torch
except ModuleNotFoundError as e:
    raise ModuleNotFoundError(
        "transformers or torch is not installed. Please run the cell with '%pip install transformers torch' at the top of the notebook, then restart your cluster or re-run all cells."
    ) from e

# Load pre-trained model and tokenizer
try:
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
except Exception as e:
    print("Error loading BERT model or tokenizer:", e)
    raise

# Function to get BERT embeddings
def get_bert_embeddings(text):
    # Encode the text
    inputs = tokenizer(text, return_tensors='pt')
    # Get the embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    # Return the last hidden state
    return outputs.last_hidden_state

# Example usage
text = "Hello, how are you?"
embeddings = get_bert_embeddings(text)
print(embeddings)

tensor([[[-0.0824,  0.0667, -0.2880,  ..., -0.3566,  0.1960,  0.5381],
         [ 0.0310, -0.1448,  0.0952,  ..., -0.1560,  1.0151,  0.0947],
         [-0.8935,  0.3240,  0.4184,  ..., -0.5498,  0.2853,  0.1149],
         ...,
         [-0.2812, -0.8531,  0.6912,  ..., -0.5051,  0.4716, -0.6854],
         [-0.4429, -0.7820, -0.8055,  ...,  0.1949,  0.1081,  0.0130],
         [ 0.5570, -0.1080, -0.2412,  ...,  0.2817, -0.3996, -0.1882]]])


In [0]:
# Assume df is your Spark DataFrame
splits = df.randomSplit([0.7, 0.15, 0.15], seed=42)
train_df = splits[0]
validation_df = splits[1]
test_df = splits[2]
# Now you can write to ADLS using the same code


In [0]:
# The following code is commented out because the path may be empty or missing and would cause schema inference errors.
# df = spark.read.parquet(
#   "abfss://lakehouse@goodreadsreviews60313569.dfs.core.windows.net/gold/feature_v2"
# )
# Using df loaded from gold/feature_v2/ instead.


In [0]:
# Use the DataFrame loaded from gold/feature_v2/
# (Assume df is already loaded in cell 3)

if not df.head(1):
    print("WARNING: DataFrame is empty. Skipping split and write steps.")
else:
    # Split the DataFrame
    total = df.count()
    train_df, validation_df, test_df = df.randomSplit([0.7, 0.15, 0.15], seed=42)

    # Save each split to its respective subfolder in the gold layer using base_path
    train_df.write.mode("overwrite").parquet(f"{base_path}/train")
    validation_df.write.mode("overwrite").parquet(f"{base_path}/validation")
    test_df.write.mode("overwrite").parquet(f"{base_path}/test")

    # Show counts for confirmation
    print(f"Total rows: {total}")
    print(f"Train rows: {train_df.count()}")
    print(f"Validation rows: {validation_df.count()}")
    print(f"Test rows: {test_df.count()}")

In [0]:
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType
from pyspark.sql import Row
import numpy as np

# Sample a subset for demonstration (adjust n as needed)
sample_df = df.select("id", "review_text").limit(1000).toPandas()

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Function to get BERT [CLS] embedding for a text
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128, padding="max_length")
    with torch.no_grad():
        outputs = model(**inputs)
        # Use the [CLS] token embedding
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return cls_embedding.astype(np.float32)

# Compute BERT embeddings for each review_text
sample_df["bert_features"] = sample_df["review_text"].fillna("").apply(get_bert_embedding)

bert_pdf = sample_df[["id", "bert_features"]]

if bert_pdf.empty:
    print("WARNING: No data in sample_df. Creating empty Spark DataFrame with schema.")
    bert_schema = StructType([
        StructField("id", StringType(), True),
        StructField("bert_features", ArrayType(FloatType()), True)
    ])
    bert_sdf = spark.createDataFrame([], schema=bert_schema)
else:
    # Convert numpy arrays to lists for Spark compatibility
    bert_pdf["bert_features"] = bert_pdf["bert_features"].apply(lambda x: x.tolist())
    bert_sdf = spark.createDataFrame(bert_pdf)

# Join BERT features back to the original DataFrame (on 'id')
result_df = df.join(bert_sdf, on="id", how="left")

# Display the DataFrame with BERT features (show a few columns)
display(result_df.select("review_text", "bert_features"))

# NOTE: For large datasets, consider using UDFs or batch processing for scalability.

review_text,bert_features


In [0]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import numpy as np
import nltk
from nltk import ngrams
import torch
from transformers import BertTokenizer, BertModel
from nltk.sentiment import SentimentIntensityAnalyzer

# Ensure vader_lexicon is downloaded
nltk.download('vader_lexicon')

# This is a pandas DataFrame for demonstration. Do NOT overwrite the Spark DataFrame 'df'.
data = {
    'reviews': [
        "I love this product! It's amazing.",
        "Worst purchase I've ever made.",
        "It's okay, not great but not terrible.",
        "Absolutely fantastic! Highly recommend.",
        "Do not waste your money on this."
    ]
}

pdf = pd.DataFrame(data)

# Function to calculate sentiment
def calculate_sentiment(reviews):
    sia = SentimentIntensityAnalyzer()
    return [sia.polarity_scores(review)['compound'] for review in reviews]

# Function to calculate review length
def review_length(reviews):
    return [len(review.split()) for review in reviews]

# Function to extract BERT embeddings
def get_bert_embeddings(reviews):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    inputs = tokenizer(reviews, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_features = tfidf_vectorizer.fit_transform(pdf['reviews']).toarray()

# N-grams
count_vectorizer = CountVectorizer(ngram_range=(1, 2))
ngram_features = count_vectorizer.fit_transform(pdf['reviews']).toarray()

# Assemble features into a DataFrame
pdf['sentiment'] = calculate_sentiment(pdf['reviews'])
pdf['review_length'] = review_length(pdf['reviews'])
bert_embeddings = get_bert_embeddings(pdf['reviews'].tolist())
bert_df = pd.DataFrame(bert_embeddings, columns=[f'bert_{i}' for i in range(bert_embeddings.shape[1])])

# Combine all features
features_df = pd.concat([pdf, pd.DataFrame(tfidf_features), pd.DataFrame(ngram_features), bert_df], axis=1)

# Display a sample for modeling
print(features_df.sample(3))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/spark-e26db6c5-ed21-4f5e-bbc3-b5/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                                   reviews  sentiment  ...  bert_766  bert_767
3  Absolutely fantastic! Highly recommend.     0.8043  ...  0.416754 -0.025454
0       I love this product! It's amazing.     0.8516  ...  0.066335 -0.029567
1           Worst purchase I've ever made.    -0.6249  ...  0.225471 -0.151352

[3 rows x 842 columns]


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Define the schema (should match Cell 5)
schema = StructType([
    StructField("id", StringType(), True),
    StructField("review_text", StringType(), True),
    StructField("rating", IntegerType(), True),
    StructField("user_id", StringType(), True),
    StructField("book_id", StringType(), True),
    StructField("bert_features", StringType(), True),
    # Add more fields as needed
])

# 1. Set up Azure Data Lake Storage access (if not already set)
account_name = "goodreadsreviews60313569"
account_key = "TLLUqPCW94aCOPP/8ic0S/0oWOEa9ccZ/rfHpUw71lh9c2vsCCl2oFGaWifNoAmtRTC45MDRTn0B+AStXqUGDQ=="

spark.conf.set(
    f"fs.azure.account.key.{account_name}.dfs.core.windows.net",
    account_key
)

# 2. Define the path to the Gold dataset (feature_v2/train)
parquet_path = (
    f"abfss://lakehouse@{account_name}.dfs.core.windows.net/gold/feature_v2/train"
)

# 3. Load the dataset into a Spark DataFrame
try:
    df = spark.read.schema(schema).parquet(parquet_path)
    if df.rdd.isEmpty():
        print(f"WARNING: Parquet path {parquet_path} is empty. Creating empty DataFrame with schema.")
        df = spark.createDataFrame([], schema)
except Exception as e:
    print(f"WARNING: Could not read Parquet path {parquet_path}. Creating empty DataFrame with schema. Error: {e}")
    df = spark.createDataFrame([], schema)

# 4. Display a sample of the review_text column
display(df.select("review_text").limit(10))

review_text


In [0]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Sample DataFrame
data = {
    'review_text': [
        'This product is great!',
        'I did not like this product.',
        'Amazing quality and service.',
        'Worst experience ever.',
        'I would recommend this to everyone.'
    ]
}
df = pd.DataFrame(data)

# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english', ngram_range=(1, 2))

# Fit and transform the review_text column
tfidf_matrix = vectorizer.fit_transform(df['review_text'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Concatenate the original DataFrame with the TF-IDF features
result_df = pd.concat([df, tfidf_df], axis=1)

# Display a sample of the resulting DataFrame
print(result_df.sample())

                    review_text  amazing  ...  worst  worst experience
1  I did not like this product.      0.0  ...    0.0               0.0

[1 rows x 17 columns]


In [0]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer

# Sample DataFrame
data = {
    'review_text': [
        'I love this product! It works great.',
        'This is the worst purchase I have ever made.',
        'It is okay, not the best but not the worst.',
        'Absolutely fantastic! Highly recommend it.',
        'I am disappointed with the quality.'
    ]
}

df = pd.DataFrame(data)

# Initialize VADER SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Function to extract sentiment features
def extract_sentiment_features(text):
    sentiment = sia.polarity_scores(text)
    return pd.Series({
        'sentiment_pos': sentiment['pos'],
        'sentiment_neg': sentiment['neg'],
        'sentiment_neu': sentiment['neu'],
        'sentiment_compound': sentiment['compound']
    })

# Apply the function to the review_text column
sentiment_features = df['review_text'].apply(extract_sentiment_features)

# Concatenate the sentiment features to the original DataFrame
df = pd.concat([df, sentiment_features], axis=1)

# Display a sample of the DataFrame with the new sentiment features
print(df.sample(5))

                                    review_text  ...  sentiment_compound
0          I love this product! It works great.  ...              0.8622
2   It is okay, not the best but not the worst.  ...              0.5729
3    Absolutely fantastic! Highly recommend it.  ...              0.8043
4           I am disappointed with the quality.  ...             -0.4767
1  This is the worst purchase I have ever made.  ...             -0.6249

[5 rows x 5 columns]


In [0]:
from pyspark.sql.functions import col, length, split, size

# Use the existing df loaded earlier (from the Gold layer)
# If you want to test with a sample DataFrame, uncomment the following lines:
data = [("This is a great product.",), ("Not what I expected.",), ("Absolutely fantastic!",)]
df = spark.createDataFrame(data, ["review_text"])

# Extracting basic text features
df = df.withColumn("review_length_words", size(split(col("review_text"), " "))).withColumn("review_length_chars", length(col("review_text")))

# Display a sample of the resulting DataFrame
display(df)

review_text,review_length_words,review_length_chars
This is a great product.,5,24
Not what I expected.,4,20
Absolutely fantastic!,2,21


In [0]:
import pandas as pd
import re
import emoji

# This is a pandas DataFrame for demonstration. Do NOT overwrite the Spark DataFrame 'df'.
data = {
    'review_text': [
        "I love this product! 😊 Check it out at https://example.com",
        "Terrible service... would not recommend!!!",
        "Great value for money. 100% satisfied.",
        "😡 I am so angry!!!",
        "Short",
        "   This is a    great product!   "
    ]
}

pdf = pd.DataFrame(data)

def clean_review(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Replace URLs
    text = re.sub(r'http\S+|www\S+|https\S+', 'URL', text, flags=re.MULTILINE)
    # Replace numbers
    text = re.sub(r'\d+', 'NUMBER', text)
    # Replace emojis
    text = emoji.replace_emoji(text, replace='EMOJI')
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply cleaning function
pdf['cleaned_review'] = pdf['review_text'].apply(clean_review)

# Filter out reviews with less than 10 characters
pdf_filtered = pdf[pdf['cleaned_review'].str.len() >= 10]

# Display a sample of the cleaned reviews
print(pdf_filtered[['review_text', 'cleaned_review']].sample(n=3, random_state=1))

                                  review_text                          cleaned_review
2      Great value for money. 100% satisfied.  great value for money NUMBER satisfied
1  Terrible service... would not recommend!!!    terrible service would not recommend
5              This is a    great product!                    this is a great product


In [0]:
%sql
CREATE TABLE IF NOT EXISTS hive_metastore.default.semantic_embeddings (
  bert_embedding ARRAY<DOUBLE> COMMENT 'Contextual vector representation for each review capturing meaning beyond surface words.'
);

CREATE OR REPLACE VIEW hive_metastore.default.semantic_embedding_features_metric_view AS
SELECT
  bert_embedding
FROM
  hive_metastore.default.semantic_embeddings;

In [0]:
# Display the available columns
display(result_df.columns)

# Display the first 5 rows with additional features if present
cols_to_show = [
    'review_text',
    'review_length_words',
    'review_length_chars',
    'sentiment_pos',
    'sentiment_neg',
    'sentiment_neu',
    'sentiment_compound'
]
display(
    result_df[
        [col for col in cols_to_show if col in result_df.columns]
    ].head(5)
)

Index(['review_text', 'amazing', 'amazing quality', 'did', 'did like',
       'experience', 'great', 'like', 'like product', 'product',
       'product great', 'quality', 'quality service', 'recommend', 'service',
       'worst', 'worst experience'],
      dtype='object')

review_text
This product is great!
I did not like this product.
Amazing quality and service.
Worst experience ever.
I would recommend this to everyone.


In [0]:
%python
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, DoubleType
from nltk.sentiment import SentimentIntensityAnalyzer

# Broadcast the analyzer for efficiency
sia = SentimentIntensityAnalyzer()
def get_sentiment_scores(text):
    scores = sia.polarity_scores(text if text else "")
    return (float(scores['pos']), float(scores['neg']), float(scores['neu']), float(scores['compound']))

schema = StructType([
    StructField("sentiment_pos", DoubleType(), True),
    StructField("sentiment_neg", DoubleType(), True),
    StructField("sentiment_neu", DoubleType(), True),
    StructField("sentiment_compound", DoubleType(), True)
])

sentiment_udf = udf(get_sentiment_scores, schema)

# Add sentiment columns
df = df.withColumn("sentiment", sentiment_udf(df["review_text"]))
df = df.withColumn("sentiment_pos", df["sentiment"].sentiment_pos) \
       .withColumn("sentiment_neg", df["sentiment"].sentiment_neg) \
       .withColumn("sentiment_neu", df["sentiment"].sentiment_neu) \
       .withColumn("sentiment_compound", df["sentiment"].sentiment_compound) \
       .drop("sentiment")

display(df)

review_text,review_length_words,review_length_chars,sentiment_pos,sentiment_neg,sentiment_neu,sentiment_compound
This is a great product.,5,24,0.577,0.0,0.423,0.6249
Not what I expected.,4,20,0.0,0.0,1.0,0.0
Absolutely fantastic!,2,21,0.807,0.0,0.193,0.6352


In [0]:
%python
from pyspark.sql.functions import array

final_df = df.withColumn(
    "features",
    array(
        "review_length_words",
        "review_length_chars",
        "sentiment_pos",
        "sentiment_neg",
        "sentiment_neu",
        "sentiment_compound"
    )
)

final_features_df = final_df.select("features")

final_features_df.write.mode("overwrite").parquet(
    "abfss://lakehouse@goodreadsreviews60313569.dfs.core.windows.net/gold/features_v2/"
)